# 클리앙 게시판의 다수 페이지, 게시글 제목 , 게시글 내용 수집
작성자 : 장지원  
날짜 : 2020.1.19  

- selenium driver를 이용하여 구현
- 수집하고자 하는 게시글 목록 생성 후 게시글 내용이 포함되어 있는 콘텐츠 링크 수집

[추후 과제]
- for loop를 list comprehension으로 표현하는 방법 고민
- 대상 서버에 최소한의 영향을 주는 방식 고민

In [17]:
from tqdm import tqdm # loop 상태 표시바 생성. 없는 경우 pip install tqdm 으로 다운로드

In [18]:
from selenium import webdriver # chrome browser
from bs4 import BeautifulSoup # beautifulsoup
from random import randint # random number generation
from time import sleep # sleep
import requests # requests.get
import pandas as pd # pandas dataframe
import os # os.getcwd()

In [19]:
# Firefox driver user
driverDir = '/Users/jiwonjang/python_tutorial/geckodriver' 
driver = webdriver.Firefox(executable_path=driverDir)

In [12]:
# chrome driver 사용자의 경우
# driver = webdriver.Chrome()

In [13]:
# 게시판 url 목록 만들기
baseUrl = 'https://www.clien.net/service/group/allinfo?&od=T31&po='
endPage = 5

In [14]:
PostPageUrl = []
for i in range(0, endPage):
    tmpUrl = baseUrl+str(i)
    PostPageUrl.append(tmpUrl)

In [15]:
FINALDF = pd.DataFrame(columns=['Title','View','Date','postUrl','Contents'])

In [20]:
for url in tqdm(PostPageUrl):
    CLIEN = pd.DataFrame(columns=['Title','View','Date','postUrl','Contents'])
    driver.get(url)
    driver.implicitly_wait(1)
    postList = driver.find_elements_by_class_name('subject_fixed') # post title
    
    Post = []
    for i in postList:
        tmp = i.get_attribute('title')
        Post.append(tmp)

    viewCount = driver.find_elements_by_class_name('hit') # post viewcount
    
    View = []
    for i in viewCount:
        View.append(i.text)

    timeStamp = driver.find_elements_by_class_name('list_time') # time
    
    PostDate = []
    for i in timeStamp:
        PostDate.append(i.text)

    ViewCount = View[2:]
    UploadDate = PostDate[2:]

    print('수집된 게시글 수:',len(Post))
    print('수집된 조회수 수:',len(ViewCount))
    print('수집된 날짜 수:',len(UploadDate))
    
    ### post contents
    ContentsHref = driver.find_elements_by_class_name('list_subject')
    Hrefs = []
    for i in ContentsHref:
        Hrefs.append(i.get_attribute('href'))
    Hrefs = Hrefs[2:]
    
    print('수집된 게시글 링크수:',len(Hrefs))
    Contents = []
    for i in Hrefs:
        driver.get(i)
        postContents = driver.find_element_by_class_name('post_content').text
        Contents.append(postContents)
        driver.implicitly_wait(randint(1,3)) # sleep or implicitly_wait : 하나만 선택할 것 
    
    print('----------------------------------------')
    #### dataframe
    CLIEN['Title'] = Post
    CLIEN['View'] = ViewCount
    CLIEN['Date'] = UploadDate
    CLIEN['postUrl'] = Hrefs
    CLIEN['Contents'] = Contents
    FINALDF = FINALDF.append(CLIEN)
    driver.implicitly_wait(randint(1,5))
print('loop finish!')

  0%|          | 0/5 [00:00<?, ?it/s]

수집된 게시글 수: 30
수집된 조회수 수: 30
수집된 날짜 수: 30
수집된 게시글 링크수: 30


KeyboardInterrupt: 

In [21]:
FINALDF

,Title,View,Date,postUrl,Contents
